### Set Project Root

In [2]:
from constants import set_cwd_to_project_root

set_cwd_to_project_root()

PAIR = "BTC/USDT:USDT"

### Initialize Datahandler

In [3]:
# Load data using values set above
from freqtrade.data.history import get_datahandler

from freqtrade.enums import CandleType
from constants import DATA_DIR

data_handler = get_datahandler(
    datadir=DATA_DIR / "binance",
    data_format="feather",
)

### Load Trades and Candles

In [5]:
from freqtrade.data.converter import populate_dataframe_with_trades
from collections import OrderedDict

# Initialize an empty OrderedDict for cached_grouped_trades
cached_grouped_trades = OrderedDict()

dataframe, cached_grouped_trades = populate_dataframe_with_trades(
    cached_grouped_trades=cached_grouped_trades,
    config=dict(
        timeframe="5m", 
        orderflow=dict(
            cache_size=1000,
            max_candles=10000,
            scale=0.5,
            stacked_imbalance_range=3,
            imbalance_volume=1,
            imbalance_ratio=3
        )
    ),
    dataframe=data_handler.ohlcv_load(PAIR, "5m", CandleType.FUTURES),
    trades=data_handler.trades_load(PAIR, CandleType.FUTURES)
).dropna(subset="delta")

dataframe.head()

### Populate indicators

In [ ]:
from pandas import DataFrame

dataframe["day"] = dataframe["date"].dt.date

dataframe.set_index("date", inplace=True)

def _calculations(group: DataFrame) -> DataFrame:
    # Calculate CVD and related indicators
    group["cvd"] = group["delta"].cumsum()

    return group

dataframe = dataframe.groupby("day").apply(_calculations).reset_index(level=0, drop=True)

In [ ]:
import numpy as np
from pandas import Series

# Add new hyperparameters
lookback = default=50
threshold = 2.0
roc_period = 5

def detect_divergences(df: DataFrame):
    """
    Detect both bullish and bearish divergences using a quantitative approach.
    """

    # Calculate rate of change
    df["price_roc"] = df["close"].pct_change(roc_period)
    df["cvd_roc"] = df["cvd"].pct_change(roc_period)

    bullish_div = np.zeros(len(df), dtype=bool)
    bearish_div = np.zeros(len(df), dtype=bool)

    for i in range(lookback, len(df)):
        price_window = df["close"].iloc[i - lookback : i]
        cvd_window = df["cvd"].iloc[i - lookback : i]

        # Find local min/max
        price_min_idx = price_window.idxmin()
        price_max_idx = price_window.idxmax()
        cvd_min_idx = cvd_window.idxmin()
        cvd_max_idx = cvd_window.idxmax()

        # Calculate slopes
        price_slope = (df["close"].iloc[i] - price_window.iloc[0]) / lookback
        cvd_slope = (df["cvd"].iloc[i] - cvd_window.iloc[0]) / lookback

        # Detect bullish divergence
        if (
            price_min_idx == price_window.index[-1]
            and cvd_min_idx != cvd_window.index[-1]
            and price_slope < 0
            and cvd_slope > 0
            and df["price_roc"].iloc[i] < -threshold
            and df["cvd_roc"].iloc[i] > threshold
        ):
            bullish_div[i] = True

        # Detect bearish divergence
        if (
            price_max_idx == price_window.index[-1]
            and cvd_max_idx != cvd_window.index[-1]
            and price_slope > 0
            and cvd_slope < 0
            and df["price_roc"].iloc[i] > threshold
            and df["cvd_roc"].iloc[i] < -threshold
        ):
            bearish_div[i] = True

    return Series(bullish_div, index=df.index), Series(bearish_div, index=df.index)

bullish_div, bearish_div = detect_divergences(dataframe)

dataframe.loc[
    (bullish_div)
    & (dataframe["close"] < dataframe["vwap"])
    & (dataframe["low"] > dataframe["bear_poc_upper"])
    & (dataframe["rsi"] < 30),
    ["enter_long", "enter_tag"],
] = (1, "Long_Divergence")

dataframe.loc[
    (bearish_div)
    & (dataframe["close"] > dataframe["vwap"])
    & (dataframe["high"] < dataframe["bull_poc_lower"])
    & (dataframe["rsi"] > 70),
    ["enter_short", "enter_tag"],
] = (1, "Short_Divergence")

dataframe.head()

### Imports

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

### Add features for model training

In [ ]:
dataframe['price_change'] = dataframe['close'].pct_change()
dataframe['cvd_change'] = dataframe['cvd'].pct_change()
dataframe['price_cvd_diff'] = dataframe['close'] - dataframe['cvd']

### Define target variable (1 for divergence, 0 for no divergence)

In [ ]:
dataframe['divergence'] = (dataframe['price_change'] * dataframe['cvd_change'] < 0).astype(int)

# Drop NaN values
dataframe.dropna(inplace=True)

### Features and target

In [ ]:
X = dataframe[['price_change', 'cvd_change', 'price_cvd_diff']]
y = dataframe['divergence']

### Split the data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Train the model

In [ ]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

### Predict and evaluate

In [ ]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

f"Model Accuracy: {accuracy:.2f}"

### Strategy Integration

In [ ]:
def detect_divergence_with_ml(dataframe: pd.DataFrame) -> pd.Series:
    features = dataframe[['price_change', 'cvd_change', 'price_cvd_diff']].fillna(0)
    predictions = model.predict(features)
    return pd.Series(predictions, index=dataframe.index)

# Integrate into your strategy
dataframe['ml_divergence'] = detect_divergence_with_ml(dataframe)

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create the candlestick chart
fig = go.Figure(
    data=[
        go.Candlestick(
            x=dataframe.index,
            open=dataframe['open'],
            high=dataframe['high'],
            low=dataframe['low'],
            close=dataframe['close']
        )
])

# Add ML divergence markers
divergence_points = dataframe[dataframe['ml_divergence'] == 1]

fig.add_trace(go.Scatter(
    x=divergence_points.index,
    y=divergence_points['close'],
    mode='markers',
    marker=dict(
        size=10,
        symbol='star',
        color='red',
        line=dict(width=2, color='DarkSlateGrey')
    ),
    name='ML Divergence'
))

# Add CVD subplot
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1, row_heights=[0.7, 0.3])

# Add candlestick chart to the first subplot
fig.add_trace(
    go.Candlestick(
        x=dataframe.index,
        open=dataframe['open'],
        high=dataframe['high'],
        low=dataframe['low'],
        close=dataframe['close'],
        name='Candlesticks'
    ),
    row=1, col=1
)

# Add ML divergence markers to the first subplot
fig.add_trace(
    go.Scatter(
        x=divergence_points.index,
        y=divergence_points['close'],
        mode='markers',
        marker=dict(
            size=10,
            symbol='star',
            color='red',
            line=dict(width=2, color='DarkSlateGrey')
        ),
        name='ML Divergence'
    ),
    row=1, col=1
)

# Add CVD to the second subplot
fig.add_trace(
    go.Scatter(
        x=dataframe.index,
        y=dataframe['cvd'],
        mode='lines',
        name='CVD'
    ),
    row=2, col=1
)

# Update layout for both subplots
fig.update_layout(
    title=f'{PAIR} OHLCV Chart with Pivot Points and CVD',
    yaxis_title='Price',
    xaxis_title='Date',
    xaxis_rangeslider_visible=False,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    height=800  # Increase the overall height to accommodate the subplot
)

# Update y-axis labels
fig.update_yaxes(title_text="Price", row=1, col=1)
fig.update_yaxes(title_text="CVD", row=2, col=1)


# Update layout
fig.update_layout(
    title=f'{PAIR} OHLCV Chart with Pivot Points',
    yaxis_title='Price',
    xaxis_title='Date',
    xaxis_rangeslider_visible=False,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

# Show the plot
fig.show()